In [48]:
import pandas as pd
import datetime as dt

In [49]:
df_spy = pd.read_csv('raw_data/SPY_data.csv', parse_dates = True, infer_datetime_format = True)

In [50]:
df_spy['timestamp'] = pd.to_datetime(df_spy['timestamp']).dt.date
df_spy['timestamp'] = pd.to_datetime(df_spy['timestamp'])
df_spy['timestamp'].dtype
df_spy.set_index('timestamp', inplace=True)

In [51]:
df_vix = pd.read_csv('raw_data/VIX_data.csv', parse_dates= True, infer_datetime_format= True)
df_vix['DATE'] = pd.to_datetime(df_vix['DATE'])
df_vix['DATE'].dtype
df_vix.set_index('DATE', inplace=True)

In [52]:
merge = pd.concat([df_spy, df_vix], axis=1, join= 'inner')

In [53]:
merge

,Unnamed: 0,open,high,low,close,volume,trade_count,vwap,OPEN,HIGH,LOW,CLOSE
2015-12-01,0,209.42,210.8200,209.1100,210.68,97858418,337780,209.921410,15.61,16.34,14.63,14.67
2015-12-02,1,210.60,211.0000,208.2300,208.54,108069059,367013,209.563055,15.04,16.49,14.71,15.91
2015-12-03,2,208.90,209.1500,204.7511,205.58,166224154,546768,206.878936,15.87,19.35,15.86,18.11
2015-12-04,3,206.10,209.9700,205.9300,209.66,192878747,556731,208.178631,17.43,17.65,14.69,14.81
2015-12-07,4,209.20,209.7295,207.2000,208.27,102027111,374705,208.276128,15.65,17.18,15.58,15.84
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-23,1588,446.90,448.4900,443.7100,443.80,77982935,627281,446.058683,23.04,24.03,22.64,23.57
2022-03-24,1589,445.82,450.5000,444.7600,450.48,61411569,526865,447.609014,23.61,23.79,21.49,21.67
2022-03-25,1590,451.09,452.9800,448.4300,452.69,77199328,661033,451.047221,21.87,22.86,20.80,20.81
2022-03-28,1591,452.04,455.9100,450.0600,455.91,68356914,620868,453.294974,22.14,23.33,19.54,19.63
